In [1]:
import pandas as pd
import re
import json
import glob
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
files = glob.glob("Clean_data/*.csv")

csv = []

for file in files:
    df = pd.read_csv(file, index_col=None, header=0)
    csv.append(df)

df = pd.concat(csv, axis=0, ignore_index=True)

In [3]:
df.head()

,Unnamed: 0,backers_count,blurb,category,converted_pledged_amount,country,created_at,current_currency,deadline,fx_rate,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,1,1,With your help we will create this device that...,"{""id"":331,""name"":""3D Printing"",""slug"":""technol...",1,ES,2015-08-18 21:01:19,USD,2016-07-09 20:11:23,1.212886,...,save-water-100-liquid-downloads-in-toilets,https://www.kickstarter.com/discover/categorie...,False,False,failed,2016-07-09 20:11:24,1.136930,"{""web"":{""project"":""https://www.kickstarter.com...",1.136930,domestic
1,2,2,We at Ormiston Primary are looking at starting...,"{""id"":309,""name"":""Farms"",""slug"":""food/farms"",""...",9,NZ,2015-08-11 18:04:21,USD,2015-09-11 15:55:41,0.723585,...,ormiston-primary-community-garden,https://www.kickstarter.com/discover/categorie...,False,False,failed,2015-09-11 15:55:41,0.655304,"{""web"":{""project"":""https://www.kickstarter.com...",9.829555,domestic
2,3,0,Self-taught aspiring metalsmith Looking for he...,"{""id"":54,""name"":""Mixed Media"",""slug"":""art/mixe...",0,US,2015-04-28 21:14:03,USD,2015-05-28 21:14:00,1.000000,...,aspiring-metalsmith-in-need-of-better-tools-an...,https://www.kickstarter.com/discover/categorie...,False,False,failed,2015-05-28 21:14:04,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",0.000000,domestic
3,4,0,So many women believe they are past their prim...,"{""id"":278,""name"":""People"",""slug"":""photography/...",0,US,2014-07-07 01:30:52,USD,2014-10-26 00:00:48,1.000000,...,beauty-at-any-age,https://www.kickstarter.com/discover/categorie...,False,False,failed,2014-10-26 00:00:48,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",0.000000,domestic
4,5,10,The Horror Zine's Jeani Rector brings us anoth...,"{""id"":324,""name"":""Anthologies"",""slug"":""publish...",340,US,2014-11-04 16:30:58,USD,2014-12-09 09:20:35,1.000000,...,shrieks-and-shivers-from-the-horror-zine,https://www.kickstarter.com/discover/categorie...,False,False,failed,2014-12-09 09:20:35,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",340.000000,international


In [6]:
print(df.shape, "\n",df.columns)

(195933, 27) 
 Index(['Unnamed: 0', 'backers_count', 'blurb', 'category',
       'converted_pledged_amount', 'country', 'created_at', 'current_currency',
       'deadline', 'fx_rate', 'goal', 'id', 'launched_at', 'location', 'name',
       'pledged', 'profile', 'slug', 'source_url', 'spotlight', 'staff_pick',
       'state', 'state_changed_at', 'static_usd_rate', 'urls', 'usd_pledged',
       'usd_type'],
      dtype='object')


In [4]:
urls = df.urls

urls_ = list(map(lambda url:json.loads(url)["web"]["project"],urls))
urls = urls_

In [6]:
### using beautifulsoup instead

#from selenium.common.exceptions import TimeoutException
#from selenium.webdriver.common.action_chains import ActionChains

#from webdriver_manager.chrome import ChromeDriverManager
#driver = webdriver.Chrome(ChromeDriverManager().install())

#chromeOptions = webdriver.ChromeOptions() 
#chromeOptions.add_argument("--no-sandbox") 
#chromeOptions.add_argument("--headless") 

#driver = webdriver.Chrome("D:/NUS/Year4_SEM2/BT4222/Grp_project/BT4222_GrpProject-main/Webdriver/chromedriver88.exe", chrome_options=chromeOptions)

In [70]:
len(urls)

195933

In [78]:
#Edit to FAQ urls

faq_urls = []

for i in range(0, len(urls)):
    split_string = urls[i].split("?", 1)
    faq = split_string[0] + "/faqs"
    faq_urls.append(faq)

In [80]:
faq_urls[-9:]

['https://www.kickstarter.com/projects/compagnienarcisse/no-borders/faqs',
 'https://www.kickstarter.com/projects/curiouserseries/book-ii-eat-me-drink-me/faqs',
 'https://www.kickstarter.com/projects/2045956967/cat-with-hat/faqs',
 'https://www.kickstarter.com/projects/kawaiikatstore/mental-health-awareness-enamel-pins/faqs',
 'https://www.kickstarter.com/projects/1854710561/nihilist-birthday-balloons/faqs',
 'https://www.kickstarter.com/projects/991561780/how-to-make-friends-as-an-adult-the-book/faqs',
 'https://www.kickstarter.com/projects/johnjennison/this-deck-is-haunted-an-oracle-of-ghost-and-spirits/faqs',
 'https://www.kickstarter.com/projects/515267715/mars-awaits-2015-calendar/faqs',
 'https://www.kickstarter.com/projects/1743551060/replacing-my-stolen-equipment/faqs']

In [58]:
#Scrape using beautifulSoup

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import requests
import time

In [41]:
#Update user agent

headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})

In [47]:
#find child tags
def processChildren(children):
    result = [""]
    # for each tag process and append result
    for child in children:
        # checks valid tag type and text content
        if checkValidTags(child):
            # strip out white space and break characters
            divText = child.getText().strip().strip("\n")
            divText = divText.replace(u'\xa0', "")
            # If result is not empty append and continue
            if len(divText) > 0:
                result[0] += (divText + " ")

    return result

#check valid elements/html tags
def checkValidTags(element):
    check = True
    if element.find(text=False):
        check = False
    return check

#Get FAQ text
def findFAQ(htmlIn):
    # select the main description <div>
    divs = htmlIn.find("div", {"class": "NS_projects__faqs_section js-project-faqs"})
    result = [""]
    
    # find all tags inside of main <div>
    children = divs.findChildren()
    result = processChildren(children)
    return result

In [82]:
def scrapeFAQ(urlClean):
    
    html = urlopen(urlClean)
    soup = BeautifulSoup(html, 'lxml')

    try:
        result = re.sub("(Frequently Asked Questions)*"
                        "(Looks like there aren't any frequently asked questions yet. Ask the project creator directly\.)*"
                        "(Don't see the answer to your question?)*"
                        "(Ask the project creator directly. Ask a question)*"
                        "(Ask a question)*"
                        , ""
                        , findFAQ(soup)[0])
        FAQ_list.append([result])
        
    except Exception as e:
        print(e)
        FAQ_list.append("could not find faqs")
        print("could not find faqs")

    return results

In [83]:
#Define start and end int
start_int = 0
end_int = 100

In [84]:
#Scrape all FAQ and append to list

FAQ_list = []

for i in tqdm(range(start_int,end_int)):
    scrapeFAQ(faq_urls[i])
    time.sleep(3)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [08:44<00:00,  5.24s/it]


In [86]:
FAQ_list
len(FAQ_list)

100

In [88]:
df_export = pd.DataFrame({"index":list(range(start_int,end_int)),
                         "faq":FAQ_list}, index = None)
filename = "Update_" + str(start_int) + "-" + str(end_int)
df_export.to_csv("Output/FAQ/{}.csv".format(filename))

In [26]:
updates_all = []

for i in tqdm(range(0,5)):
    driver.get(test[i])

    # 1. Load all results
    while True:
        try:
            ActionChains(driver).move_to_element(WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Load more')]")))).pause(5).click().perform()
    #         print("LOAD MORE button clicked")
        except TimeoutException:
    #         print("No more LOAD MORE button to be clicked")
            break

    # 2. Get all updates and save as a list
    updates = []
    try: 
        elements = WebDriverWait(driver,10).until(EC.visibility_of_all_elements_located((By.XPATH,"//article")))
        for element in elements:
            updates.append(element.text)

    except TimeoutException:
        pass
    
    updates_all.append(updates)

driver.quit()

  0%|                                                                                            | 0/5 [00:08<?, ?it/s]


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=60935): Max retries exceeded with url: /session/b1b3e78cd8c91bcdb3f4214cd0f8f0e1/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001E1557E9430>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))